In [0]:
%pip install databricks-sdk --quiet --disable-pip-version-check
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
def get_cluster_id(cluster_name):
  import requests
  import json
  headers = {"Authorization": "Bearer " + dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}
  base_url = "https://" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
  endpoint = f'/api/2.0/clusters/list'

  # Make the API request
  response = requests.get(base_url+endpoint, headers=headers)
  if response.status_code == 200:
      clusters = response.json().get("clusters", [])
      for cluster in clusters:
          if cluster.get("cluster_name") == cluster_name:
                return cluster.get("cluster_id")

In [0]:
def terminate_cluster(cluster_id):
  from databricks.sdk import WorkspaceClient
  host = spark.conf.get("spark.databricks.workspaceUrl")
  token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
  cluster_id = cluster_id 
  #spark.conf.get("spark.databricks.clusterUsageTags.clusterId")
  w = WorkspaceClient(host=host, token=token)
  w.clusters.delete(cluster_id) 

In [0]:

def remove_cluster_permissions(cluster_id):
  import requests
  import json
  headers = {"Authorization": "Bearer " + dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}
  base_url = "https://" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
  payload = {
      "access_control_list": [
      ]
  }
  endpoint = f'/api/2.0/permissions/clusters/{cluster_id}'

  # Make the API request
  response = requests.put(base_url + endpoint, headers = headers, data=json.dumps(payload))

In [0]:
def set_cluster_permissions(cluster_id,group_name):
  import requests
  import json
  headers = {"Authorization": "Bearer " + dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}
  base_url = "https://" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
  payload = {
      "access_control_list": [
          {
              "group_name": group_name,
              "permission_level": "CAN_ATTACH_TO"
          }
      ]
  }
  endpoint = f'/api/2.0/permissions/clusters/{cluster_id}'

  # Make the API request
  response = requests.patch(base_url + endpoint, headers = headers, data=json.dumps(payload))
  return response.json()

In [0]:
cluster_name = dbutils.widgets.get("cluster_name")


In [0]:
cluster_id= get_cluster_id(cluster_name)

In [0]:

df = spark.sql(f'''
select
  SUM(lp.pricing.`default` * uso.usage_quantity) as consumo_atual
from
  system.billing.usage uso
  left join system.billing.list_prices lp on lp.sku_name = uso.sku_name and lp.price_end_time is NULL
  left join (
    select
      row_number() OVER (partition by cluster_id ORDER BY change_time desc) as row_number,
      cluster_id,
      cluster_name
    from
      system.compute.clusters
    order by
      cluster_id
  ) cl on cl.cluster_id = uso.usage_metadata.cluster_id and cl.row_number = 1
where
  true
  and cl.cluster_name = '{cluster_name}'
  /*and date_trunc("month", uso.usage_end_time)>= date_trunc("month", current_timestamp()) - interval '6 month'*/
  and date_trunc("month", uso.usage_end_time)>= date_trunc("month", current_timestamp()) 
group by
  all
''')



In [0]:
threshold=dbutils.widgets.get("limite_mensal")
usage = df.collect()[0][0]
#Se o consumo do mês corrente for maior que o parâmetro limite_mensal, o cluster deve ser desligado
terminate = True if usage > float(threshold) else False

print(f"The montlhy usage is {usage} and the threshold is {threshold}. Terminate = {terminate}")

The montlhy usage is 4.916481 and the threshold is 100. Terminate = False


In [0]:
if terminate:
    print("Cluster terminated")
    remove_cluster_permissions(cluster_id)
    set_cluster_permissions(cluster_id,"professores")
    terminate_cluster(cluster_id)
elif not terminate:
    print("Usage is under the limit")

Usage is under the limit
